# Speed Skydiving scoring bulk files processor
Uses the `./data` directory as a data lake.  It finds all valid FlySight files in that directory, from 0 to as many as it finds and fit in memory, and process them all.  Summary of "all files" or "all jumps" at the bottom of the document.

In [1]:
from collections import namedtuple
from copy import deepcopy

from ssscoring.constants import BREAKOFF_ALTITUDE
from ssscoring.constants import FT_IN_M
from ssscoring.constants import PERFORMANCE_WINDOW_LENGTH
from ssscoring.fs1 import aggregateResults
from ssscoring.fs1 import convertFlySight2SSScoring
from ssscoring.fs1 import dropNonSkydiveDataFrom
from ssscoring.fs1 import getAllSpeedJumpFilesFrom
from ssscoring.fs1 import getSpeedSkydiveFrom
from ssscoring.fs1 import isValidJump
from ssscoring.fs1 import isValidMinimumAltitude
from ssscoring.fs1 import jumpAnalysisTable
from ssscoring.fs1 import processAllJumpFiles
from ssscoring.fs1 import roundedAggregateResults
from ssscoring.fs1 import totalResultsFrom
from ssscoring.fs1 import validFlySightHeaderIn
from ssscoring.notebook import SPEED_COLORS
from ssscoring.notebook import graphAltitude
from ssscoring.notebook import graphAngle
from ssscoring.notebook import graphJumpResult
from ssscoring.notebook import initializeExtraYRanges
from ssscoring.notebook import initializePlot

import csv
import os
import os.path as path

import bokeh.plotting as bp
import bokeh.models as bm
import ipywidgets as widgets
import pandas as pd

In [2]:
DATA_LAKE_ROOT = './data'

---
## <span style="color: yellow">IMPORTANT</span> - Set the drop zone altitude MSL

Set the value in ft (even though SSScoring uses meters) out of convenience because all DZs in the US, many in Asia and Mexico, and many in Europe express the altitude in feet.

### Some DZs

| Drop zone | Alt (ft) |
|-----------|----------|
| Bay Area Skydiving | 23 |
| Skydive Arizona | 1509 |
| Skydive Chicago | 616 |
| Thai Sky Adventures | 21 |

In [3]:
dropZoneAltMSL =1509

In [4]:
dropZoneAltMSLMeters = dropZoneAltMSL/FT_IN_M
display(widgets.HTML('<h2>DZ Altitude = <span style = "color: green">%7.2f ft</span> (%7.2f m)<h1>' % (dropZoneAltMSL, dropZoneAltMSLMeters)))

HTML(value='<h2>DZ Altitude = <span style = "color: green">1509.00 ft</span> ( 459.95 m)<h1>')

In [5]:
jumpFiles = getAllSpeedJumpFilesFrom(DATA_LAKE_ROOT)

In [6]:
jumpResults = processAllJumpFiles(jumpFiles, altitudeDZMeters = dropZoneAltMSLMeters)

---
## Results

In [7]:
aggregate = aggregateResults(jumpResults)
sumResults = totalResultsFrom(aggregate)
display(aggregate)

,score,5.0,10.0,15.0,20.0,25.0,finalTime,maxSpeed
00-00-00-EC-baseline,483.93936,183.564,327.204,422.892,470.844,486.036,24.4,487.152
07-36-35,464.265,180.036,318.672,411.948,453.276,463.284,24.9,465.876
08-26-01,462.410667,190.692,309.204,401.868,450.072,465.516,25.0,465.516
09-34-53,481.443231,180.000,284.472,392.436,457.056,483.480,25.0,484.992
10-51-16,483.93936,183.564,327.204,422.892,470.844,486.036,24.4,487.152
12-09-17,486.99,181.944,325.512,417.276,477.504,488.088,24.4,490.140
19-22-17-ND,515.664,189.000,334.800,438.012,499.644,523.044,23.8,523.044


### Rounded results for training log

In [8]:
roundedResults = roundedAggregateResults(jumpResults)

### All jumps

In [9]:
def dumpSimple(table: pd.DataFrame):
    fileName = os.path.join(DATA_LAKE_ROOT, resultRef+'.tsv')
    simpleTable = table.copy()
    simpleTable = simpleTable.drop('speedAngle', axis = 1)
    simpleTable.to_csv(fileName, sep = '\t', index = False)

In [10]:
def displayJumpDataIn(resultsTable: pd.DataFrame):
    table = resultsTable.copy()
    # Experimental
    # For more information on the `interpolate` method and its options, see the [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.interpolate.html).
    # Additionally, you can also consider using other libraries like `scipy` which provides more advanced interpolation methods, such as `interp1d` or `griddata`. See the [scipy documentation](https://docs.scipy.org/doc/scipy/reference/interpolate.html) for more information.
    table.vKMh = table.vKMh.apply(round)
    table.hKMh = table.hKMh.apply(round)
    table['altitude (ft)'] = table['altitude (ft)'].apply(round)
    table.netVectorKMh = table.netVectorKMh.apply(round)
    table.index = ['']*len(table)
    display(table)
    # dumpSimple(table)

In [11]:
allJumpsPlot = initializePlot('All jumps in set')
jumpNumber = 0
mixColor = 0
for resultRef in sorted(jumpResults.keys()):
    jumpResult = jumpResults[resultRef]
    if jumpResult.score > 0.0:
        validJumpStatus = '<hr><h1><span style="color: %s">%s jump - %s - score = %.02f km/h</span></h1>' % (jumpResult.color, resultRef, jumpResult.result, jumpResult.score)        
    else:
        validJumpStatus = '<hr><h1><span style="color: %s">%s jump - %s</span></h1>' % (jumpResult.color, resultRef, jumpResult.result)
    display(widgets.HTML(validJumpStatus))

    maxSpeed = jumpResult.maxSpeed
    window = jumpResult.window
    mixColor = (mixColor+1)%len(SPEED_COLORS)
    if jumpResult.score > 0.0:
        display(widgets.HTML('<h3>Max speed = {0:,.0f}; '.format(maxSpeed)+('exit at %d m (%d ft), end scoring window at %d m (%d ft)'%(window.start, 3.2808*window.start, window.end, 3.2808*window.end))))
        if not isValidMinimumAltitude(jumpResult.data.altitudeAGL.max()):
            display(widgets.HTML('<span style="color: yellow"><span style="font-weight: bold">Warning:</span> exit altitude ASL was lower than the minimum scoring altitude according to IPC and USPA.'))
        display(widgets.HTML('</h3>'))
        # TODO: Move this to an in-notebook function or module.
        displayJumpDataIn(jumpResult.table)
        individualPlot = initializePlot(resultRef)
        individualPlot = initializeExtraYRanges(individualPlot, startY = min(jumpResult.data.altitudeAGLFt)-500.0, endY = max(jumpResult.data.altitudeAGLFt)+500.0)
        graphAltitude(individualPlot, jumpResult)
        graphAngle(individualPlot, jumpResult)
        hoverValue = bm.HoverTool(tooltips = [ ('Y-val', '@y{0.00}'), ])
        individualPlot.add_tools(hoverValue)
        graphJumpResult(individualPlot,
                        jumpResult,
                        lineColor = SPEED_COLORS[0])
        graphJumpResult(allJumpsPlot,
                        jumpResult,
                        lineColor = SPEED_COLORS[mixColor],
                        legend = '%s - %.2f' % (resultRef, jumpResult.score),
                        showIt = False)

HTML(value='<hr><h1><span style="color: #0f0">00-00-00-EC-baseline jump - 🟢 valid - score = 483.94 km/h</span>…

HTML(value='<h3>Max speed = 487; exit at 4189 m (13746 ft), end scoring window at 1933 m (6344 ft)')

HTML(value='</h3>')

,time,vKMh,hKMh,speedAngle,netVectorKMh,altitude (ft)
,5.0,184,160,48.9,244,13311
,10.0,327,121,69.7,349,12124
,15.0,423,92,77.7,433,10386
,20.0,471,52,83.8,474,8320
,25.0,486,23,87.3,487,6381


HTML(value='<hr><h1><span style="color: #0f0">07-36-35 jump - 🟢 valid - score = 464.27 km/h</span></h1>')

...

,time,vKMh,hKMh,speedAngle,netVectorKMh,altitude (ft)
,5.0,182,146,51.2,234,13635
,10.0,326,99,73.1,340,12445
,15.0,417,82,78.8,425,10725
,20.0,478,48,84.3,480,8653
,25.0,488,30,86.5,489,6700


HTML(value='<hr><h1><span style="color: #0f0">19-22-17-ND jump - 🟢 valid - score = 515.66 km/h</span></h1>')

HTML(value='<h3>Max speed = 523; exit at 4312 m (14149 ft), end scoring window at 2056 m (6748 ft)')

HTML(value='</h3>')

,time,vKMh,hKMh,speedAngle,netVectorKMh,altitude (ft)
,5.0,189,142,53.2,236,13705
,10.0,335,90,74.9,347,12492
,15.0,438,72,80.7,444,10711
,20.0,500,62,83.0,503,8556
,25.0,523,43,85.3,525,6778


---
## Summary of all speed skydives in the data lake

In [12]:
display(roundedResults)
display(sumResults)
bp.show(allJumpsPlot)

,score,5.0,10.0,15.0,20.0,25.0,finalTime,maxSpeed
00-00-00-EC-baseline,484,184,327,423,471,486,25.0,487
07-36-35,464,180,319,412,453,463,25.0,466
08-26-01,462,191,309,402,450,466,25.0,466
09-34-53,481,180,284,392,457,483,25.0,485
10-51-16,484,184,327,423,471,486,25.0,487
12-09-17,487,182,326,417,478,488,25.0,490
19-22-17-ND,516,189,335,438,500,523,25.0,523


,totalSpeed,meanSpeed,maxScore
totalSpeed,3378.651617,482.664517,515.664
